In [2]:
#!pip install duckdb --proxy=http://proxyvip.national.agri:8080
#!pip install httpfs --proxy=http://proxyvip.national.agri:8080

In [44]:
import pandas as pd
import geopandas as gpd
import duckdb 

In [42]:
duckdb.sql('LOAD httpfs ;INSTALL spatial; LOAD spatial;')

In [7]:
url = "http://minio.lab.sspcloud.fr/h529p3/data/Filosofi2017_carreaux_200m_met.parquet"
url = "Filosofi2017_carreaux_200m_met.parquet"

In [30]:
duckdb.sql(f'''
  CREATE OR REPLACE VIEW carreaux
  AS SELECT * FROM read_parquet("{url}") 
'''
)

In [31]:
carreaux_test = duckdb.sql(
  "SELECT * FROM carreaux USING SAMPLE 10 ROWS"
  ).to_df()

In [32]:
carreaux_test.head()

,Idcar_200m,I_est_200,Idcar_1km,I_est_1km,Idcar_nat,Groupe,Ind,Men_1ind,Men_5ind,Men_prop,...,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc,Men_pauv,Men,lcog_geo,geometry
0,CRS3035RES200mN2644400E3994400,1,CRS3035RES1000mN2644000E3994000,0,CRS3035RES8000mN2640000E3992000,663127,9.0,0.6,0.2,3.2,...,1.4,1.6,1.0,2.1,1.0,0.0,0.6,3.8,3954839434,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 59, 18..."
1,CRS3035RES200mN2937600E3784000,0,CRS3035RES1000mN2937000E3784000,0,CRS3035RES4000mN2936000E3784000,930411,128.0,19.0,3.0,35.0,...,22.0,33.0,11.0,11.0,5.0,0.0,7.0,56.0,60509,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 121, 1..."
2,CRS3035RES200mN2347400E3567000,1,CRS3035RES1000mN2347000E3567000,1,CRS3035RES2000mN2346000E3566000,204905,3.0,0.5,0.1,1.3,...,0.2,0.4,0.7,0.8,0.4,0.0,0.2,1.5,32132,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 170, 4..."
3,CRS3035RES200mN2566400E4028400,1,CRS3035RES1000mN2566000E4028000,0,CRS3035RES1000mN2566000E4028000,626653,19.5,2.6,0.6,6.8,...,2.7,4.1,3.7,2.0,1.0,0.0,0.6,7.8,74185,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 132, 1..."
4,CRS3035RES200mN2622800E3517600,1,CRS3035RES1000mN2622000E3517000,0,CRS3035RES4000mN2620000E3516000,652162,13.0,0.9,0.1,4.1,...,3.3,2.9,1.2,1.0,0.6,0.1,0.4,5.2,79166,"[1, 3, 0, 0, 0, 1, 0, 0, 0, 5, 0, 0, 0, 165, 8..."


Requetage des carreaux à 1km de notre dame

In [48]:
latitude = 48.853
longitude = 2.35
rayon = 1000

In [51]:
duckdb.sql(
f'''select * from carreaux, (SELECT ST_BUFFER(ST_Transform(ST_Point({latitude} , {longitude}),'EPSG:4326', 'EPSG:2154'),{rayon}) AS hubblo) '''\
"where st_intersects(ST_GEOMFROMWKB(carreaux.geometry), hubblo)"

)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────────────┬───────────┬───┬─────────────────┬──────────────────────┬──────────────────────┐
│      Idcar_200m      │ I_est_200 │ … │    lcog_geo     │       geometry       │        hubblo        │
│       varchar        │   int64   │   │     varchar     │         blob         │       geometry       │
├──────────────────────┼───────────┼───┼─────────────────┼──────────────────────┼──────────────────────┤
│ CRS3035RES200mN288…  │         0 │ … │ 75105           │ \x01\x03\x00\x00\x…  │ POLYGON ((653304.3…  │
│ CRS3035RES200mN288…  │         0 │ … │ 75105           │ \x01\x03\x00\x00\x…  │ POLYGON ((653304.3…  │
│ CRS3035RES200mN288…  │         0 │ … │ 75105           │ \x01\x03\x00\x00\x…  │ POLYGON ((653304.3…  │
│ CRS3035RES200mN288…  │         0 │ … │ 75105           │ \x01\x03\x00\x00\x…  │ POLYGON ((653304.3…  │
│ CRS3035RES200mN288…  │         0 │ … │ 75105           │ \x01\x03\x00\x00\x…  │ POLYGON ((653304.3…  │
│ CRS3035RES200mN288…  │         0 │ … │ 75105         

In [16]:
duckdb.__version__

'0.9.2'